In [3]:
import urllib
from bs4 import BeautifulSoup
import re 

In [4]:
from pymongo import MongoClient
client = MongoClient('mongodb://192.168.22.16:27017/')
db = client.qiushibaike
qiushibaike_article = db.qiushibaike_article

In [5]:
pattern = re.compile(r'\d+')

def spider(page): 
    url = 'https://www.qiushibaike.com/hot/page/' + str(page)
    try:
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'
        headers = {"User-Agent": user_agent}
        req = urllib.request.Request(url, headers= headers)
        response = urllib.request.urlopen(req)
        soup = BeautifulSoup(response.read().decode('utf-8'), "html5lib")

        divs = soup.body.select('div[class^="article block untagged mb15"]')
        for div in divs:
            print("#"* 50)
            article = {}
            nick_name = div.find_next("div").find("img")["alt"]
            article["nick_name"] = nick_name
            print("昵称: "+ div.find_next("div").find("img")["alt"])
            if nick_name != "匿名用户":
                user_id = div.find_next("div").find("img").parent["href"]
                user_id = pattern.search(user_id)
                print("用户ID: " + str(user_id.group()))
                article["user_id"] = user_id.group()
            else:
                continue
                
            print("头像："+"http:" + div.find_next("div").find("img")["src"])
            article["avotor"] = div.find_next("div").find("img")["src"]
            
            content_list = div.select('a[class="contentHerf"]')
            content_a = None if len(content_list) == 0 else content_list[0]
            if not content_a:
                continue
            print("文章详情ID: " +  pattern.search(content_a['href']).group())
            article["_id"] = pattern.search(content_a['href']).group()
            article["simple"] = list(content_a.div.span.stripped_strings)
            print("文章简介: " + str(content_a.div.span.string))
            print(list(content_a.div.span.stripped_strings))

            for i in div.select('i[class="number"]'):
                if len(i.parent["class"]) > 0: 
                    if i.parent["class"][0] == u"stats-vote":
                        print("好笑: " + i.string)
                        article["vote"] = int(i.string)
                    else:
                        print("评论数: " + i.string)  
                        article["comment_number"] = int(i.string)
                        
            qiushibaike_article.insert_one(article)
                    
    except urllib.error.URLError as e:
        if hasattr(e, 'code'):
            print(e.code)
        if hasattr(e, 'reason'):
            print(e.reason)

In [6]:
for i in range(12):
    spider(i+1)

##################################################
昵称: 遛弯儿滴虫
用户ID: 34422153
头像：http://pic.qiushibaike.com/system/avtnew/3442/34422153/thumb/2017090709551937.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119717845
文章简介: None
['本人八零后，曾参军。昨晚酒桌上和一位七零后曾当过兵的老哥抬杠。            老兵:我当兵时素质好十公里不在话下。我:我们那时每天十公里全副武装。             老兵:我当兵时咱全县战友有四十多个，现在没事经常聚会。谁有事全来帮忙。    我:俺们那批有一百来号战友，每年聚会包间都坐不下。', '“呵呵，先走一个再继续。”                     老兵吧啦下嘴:我战友走了俩了。', '我:我战友也~~，走俩去哪了？', '老兵:一个肝癌，一个车祸。全是杯中好东西造成的。', '我默默放\n…']
好笑: 451
评论数: 9
##################################################
昵称: 胸平浪静
用户ID: 6349330
头像：http://pic.qiushibaike.com/system/avtnew/634/6349330/thumb/2017110919500176.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119732769
文章简介: None
['同学小聚，女同学说了一个励志故事 :', '“半年前，我老公还是公司的一个小职员。现在，我老公已经是这个公司的老总了。”', '众人目瞪口呆 : 他咋升的这么快？！', '她淡定的说 : 是我换了个老公！！！']
好笑: 10049
评论数: 79
##################################################
昵称: 对方正在偷人….
用户ID: 16314065
头像：http://pic.qiushibaike.com/system/avtnew/1631/16314065/thumb/2017081008335150.JPEG

##################################################
昵称: 一般人的一般
用户ID: 32553281
头像：http://pic.qiushibaike.com/system/avtnew/3255/32553281/thumb/20170104192654.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119712781
文章简介: 


昨天去家访，孩子的妈妈正在地里挖土，准备种菜，为了不耽误人家，就选择在菜地边询问一些孩子的事情，后来跟家长大概聊得比较投缘，家长就说了一件她家女儿的暖心故事：孩子的幺爸去年过世了，幺妈一时间六神无主，十分悲痛，幺妈就说，灿灿你晚上陪幺妈睡觉吧，孩子说好的，然后半夜幺妈要去上厕所，大概1点多钟，幺妈叫醒了孩子，让孩子扶她去厕所，孩子立马揉了一下眼睛，睁开，扶着幺妈去厕所，幺妈说“灿灿，你快去睡吧”，孩子说“不行，我等你出来，我要把你扶到床上去”，果然孩子等了几分钟，扶着幺妈一起睡觉去了，孩子就这样
…

['昨天去家访，孩子的妈妈正在地里挖土，准备种菜，为了不耽误人家，就选择在菜地边询问一些孩子的事情，后来跟家长大概聊得比较投缘，家长就说了一件她家女儿的暖心故事：孩子的幺爸去年过世了，幺妈一时间六神无主，十分悲痛，幺妈就说，灿灿你晚上陪幺妈睡觉吧，孩子说好的，然后半夜幺妈要去上厕所，大概1点多钟，幺妈叫醒了孩子，让孩子扶她去厕所，孩子立马揉了一下眼睛，睁开，扶着幺妈去厕所，幺妈说“灿灿，你快去睡吧”，孩子说“不行，我等你出来，我要把你扶到床上去”，果然孩子等了几分钟，扶着幺妈一起睡觉去了，孩子就这样\n…']
好笑: 504
评论数: 12
##################################################
昵称: 哆啦A梦ア
用户ID: 29314260
头像：http://pic.qiushibaike.com/system/avtnew/2931/29314260/thumb/2017110210191333.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119735282
文章简介: None
['…我哥下班回家，看到嫂子躺在沙发上玩手机，妞妞委屈的噘着嘴，跟我哥说“爸爸，你做饭吃吧，我

##################################################
昵称: 没有心的妖
用户ID: 12982513
头像：http://pic.qiushibaike.com/system/avtnew/1298/12982513/thumb/20171030110841.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119719090
文章简介: 


记得那天，送她进火车站，目送她远走的时候，真的很心碎，可是注定不能在一起，在一起上班半年多以来，每天朝夕相处，工作生活中，总是充斥着你的欢声笑语，彼此不肯说破的那一层关系，像是维系着你我之间朦胧暧昧的纽带，知道你走的前一天晚上，我说要送你，你问我为什么，沉默了一下，说 我喜欢你行了吧，你笑了，说 妈的为什么会有眼泪，听的我好感动，那一刻，好想把你抱在怀中，隔着手机屏幕，却没办法替你拭去眼角的泪水，进检票口之前，你回头说再见，我想最后也是唯一一次拥抱你下，想想还是算了，留下一点遗憾，也未尝不好，自
…

['记得那天，送她进火车站，目送她远走的时候，真的很心碎，可是注定不能在一起，在一起上班半年多以来，每天朝夕相处，工作生活中，总是充斥着你的欢声笑语，彼此不肯说破的那一层关系，像是维系着你我之间朦胧暧昧的纽带，知道你走的前一天晚上，我说要送你，你问我为什么，沉默了一下，说 我喜欢你行了吧，你笑了，说 妈的为什么会有眼泪，听的我好感动，那一刻，好想把你抱在怀中，隔着手机屏幕，却没办法替你拭去眼角的泪水，进检票口之前，你回头说再见，我想最后也是唯一一次拥抱你下，想想还是算了，留下一点遗憾，也未尝不好，自\n…']
好笑: 534
评论数: 101
##################################################
昵称: 别闹baby
用户ID: 33373264
头像：http://pic.qiushibaike.com/system/avtnew/3337/33373264/thumb/20171004220626.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119735062
文章简介: 


小时候翻窗户进屋偷老妈的钱，忽然听到外面有开门的声音，吓得拉开大衣柜门想躲进去。没想到老爸居然拿着几

##################################################
昵称: 莜妖七
用户ID: 12974753
头像：http://pic.qiushibaike.com/system/avtnew/1297/12974753/thumb/20140114175419.jpg?imageView2/1/w/90/h/90
文章详情ID: 119713161
文章简介: 


06年在沂水上高一，学校那条街附近有个阿拉丁网吧，那会才1 7没身份证，开通宵用临时卡。半夜我和同学玩游戏正嗨，突然冲进来一群抱着枪的武警！当时我就吓傻了:不就是未成年上网吗！至于抱着枪来抓吗？？！！[大哭][大哭]当时腿都吓得打摆子，一想到被抓走要通知学校和家长，心想全完了，结果武警叔叔看了我俩一眼就走了，后来才知道有个逃犯在上网，被抓走了……吓得我一个多月没敢去网吧。


['06年在沂水上高一，学校那条街附近有个阿拉丁网吧，那会才1 7没身份证，开通宵用临时卡。半夜我和同学玩游戏正嗨，突然冲进来一群抱着枪的武警！当时我就吓傻了:不就是未成年上网吗！至于抱着枪来抓吗？？！！\x01[大哭]\x01[大哭]当时腿都吓得打摆子，一想到被抓走要通知学校和家长，心想全完了，结果武警叔叔看了我俩一眼就走了，后来才知道有个逃犯在上网，被抓走了……吓得我一个多月没敢去网吧。']
好笑: 685
评论数: 23
##################################################
昵称: 大王mm
用户ID: 35147100
头像：http://pic.qiushibaike.com/system/avtnew/3514/35147100/thumb/20171106185008.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119731293
文章简介: None
['晚上拿个包裹拆包拆到这个，谁寄的？名字都没留，我找谁退去？', '这次就算了，我收下。下次再这样我就翻脸了']
好笑: 680
评论数: 82
##################################################
昵称: 老树林的雕
用户ID: 33676416
头像：http://pic.qiushibaik

##################################################
昵称: 匿名用户
##################################################
昵称: Answer﹏相依°
用户ID: 31218666
头像：http://pic.qiushibaike.com/system/avtnew/3121/31218666/thumb/2017110600100735.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119722363
文章简介: 


带侄子逛宽窄巷子，满街都是脸谱娃娃的，他见到都要喊买买买，家里已经买了两个，我就跟他说家里面有，这时候一个逗比摊主说：小朋友，你姑姑骗你的呢，你家里没有，快买一个…


['带侄子逛宽窄巷子，满街都是脸谱娃娃的，他见到都要喊买买买，家里已经买了两个，我就跟他说家里面有，这时候一个逗比摊主说：小朋友，你姑姑骗你的呢，你家里没有，快买一个…']
好笑: 1448
评论数: 31
##################################################
昵称: 吃了两碗又盛
用户ID: 32215536
头像：http://pic.qiushibaike.com/system/avtnew/3221/32215536/thumb/201711071252265.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119732884
文章简介: None
['坐哥们车，后座有两个妹子，副驾空着，但那是哥们媳妇的专座，我就奔后座去了，妹子们已经给我腾好了位置。刚要拉车门，哥们说:哎哎哎，坐副驾，我媳妇来了让她坐后面。', '切，谁稀罕后座似的。\x01[哀怨]\x01[哀怨]']
好笑: 1587
评论数: 19
##################################################
昵称: 萌宠控ing
用户ID: 30987301
头像：http://pic.qiushibaike.com/system/avtnew/3098/30987301/thumb/2017022218453741.JPEG?imageView2/1/w/90/h/90
文章详

...

##################################################
昵称: 蘅水老白干
用户ID: 22485307
头像：http://pic.qiushibaike.com/system/avtnew/2248/22485307/thumb/20141108144607.jpg?imageView2/1/w/90/h/90
文章详情ID: 119716112
文章简介: 


2017年11月3日晚上九点左右，LZ出门喝酒骑电动车去的没开车，喝的有点多，脑子有点迷糊，昏昏欲睡，在山东泰安市泰玻大街，由西向东回家。背景完毕，割了吧～～路上骑到一个坑里一颠簸没把住车把摔倒了，脸着地了[汗颜]，当时就摔懵了，趴地上没起来，开车路过的几位大哥见状立即停车去扶我把我扶到路边，看我喝多了要送我回去，让我给家里打电话！摔这一下我也立马醒酒了。。。我觉得很不好意思，连忙说我没事，谢谢几位大哥了！今天很感动！发这些不为别的，今天这事让我知道了这世上还是好人多！不知道三位哥哥玩不玩这坨
…

['2017年11月3日晚上九点左右，LZ出门喝酒骑电动车去的没开车，喝的有点多，脑子有点迷糊，昏昏欲睡，在山东泰安市泰玻大街，由西向东回家。背景完毕，割了吧～～路上骑到一个坑里一颠簸没把住车把摔倒了，脸着地了\x01[汗颜]，当时就摔懵了，趴地上没起来，开车路过的几位大哥见状立即停车去扶我把我扶到路边，看我喝多了要送我回去，让我给家里打电话！摔这一下我也立马醒酒了。。。我觉得很不好意思，连忙说我没事，谢谢几位大哥了！今天很感动！发这些不为别的，今天这事让我知道了这世上还是好人多！不知道三位哥哥玩不玩这坨\n…']
好笑: 429
评论数: 15
##################################################
昵称: 奈何桥湿盎卖碗汤
用户ID: 32168280
头像：http://pic.qiushibaike.com/system/avtnew/3216/32168280/thumb/20171014032546.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119724230
文章简介: 


我试了确实可以，准备去多申请几个号！


['我试了确实可以，准备去多申请几个号！']
好笑:

##################################################
昵称: 匿名用户
##################################################
昵称: 村支书的水萝卜
用户ID: 28831688
头像：http://pic.qiushibaike.com/system/avtnew/2883/28831688/thumb/20171101100743.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119733163
文章简介: None
['跟男友坐公交吵了几句后谁都不理谁。', '一会上来的人好多，男友给一个大爷让坐后被挤到后面去了。我耳坠掉地上，弯腰捡的时候到站了，等我起来后司机已经开车，再一看男友被挤下车，在着急的追着喊我呢！', '我说司机师傅慢点，内司机大哥慢悠悠说：怕啥，不让着你，就让他多追一会。']
好笑: 3492
评论数: 30
##################################################
昵称: 不帅但很能干的人
用户ID: 9318136
头像：http://pic.qiushibaike.com/system/avtnew/931/9318136/thumb/20160106115426.jpg?imageView2/1/w/90/h/90
文章详情ID: 119731892
文章简介: None
['高中军训是夏天，太阳下站军姿，一站就是半小时，就跟站我旁边的两个男同学商量，我装晕倒，他们两个抬我去医务室。', '当我倒下装晕时，本来教官已经同意让他俩抬我。结果，他们冲过来太急，两个人各踩我一支手上，疼的我大喊着跳了起来……', '然后，我们三个以后，每天都要多站半小时军姿！……']
好笑: 3121
评论数: 39
##################################################
昵称: 偷惢
用户ID: 32236400
头像：http://pic.qiushibaike.com/system/avtnew/3223/32236400/thumb/2017102420463750.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119

##################################################
昵称: 黄山小妖
用户ID: 20164589
头像：http://pic.qiushibaike.com/system/avtnew/2016/20164589/thumb/20170619172130.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119727493
文章简介: 


晚饭后约男朋友逛公园，好巧的是他刚拉住我的手，就被路过的老爸撞见了...男朋友紧张的说道：“叔...叔...好...”为了化解着尴尬的气氛，我笑着介绍道：“爸，这就是我总跟你提起的男生，体校的，跑的可快了！”老爸淡定的说道：“跑的快？你让他从这跑到最前面的电线杆，然后跑回来，我看看能用多长时间！”男朋友二话没说就跑了出去，为了表现自己，那叫一个快呀！累的是蹲在地上，气喘吁吁的说道：“累死...我了...叔叔...多...多少秒...”就见我老爸从身后拿出一个棒子，淡定的说道：“累了，是不...”


['晚饭后约男朋友逛公园，好巧的是他刚拉住我的手，就被路过的老爸撞见了...男朋友紧张的说道：“叔...叔...好...”为了化解着尴尬的气氛，我笑着介绍道：“爸，这就是我总跟你提起的男生，体校的，跑的可快了！”老爸淡定的说道：“跑的快？你让他从这跑到最前面的电线杆，然后跑回来，我看看能用多长时间！”男朋友二话没说就跑了出去，为了表现自己，那叫一个快呀！累的是蹲在地上，气喘吁吁的说道：“累死...我了...叔叔...多...多少秒...”就见我老爸从身后拿出一个棒子，淡定的说道：“累了，是不...”']
好笑: 1124
评论数: 27
##################################################
昵称: 王八与蛋
用户ID: 16749385
头像：http://pic.qiushibaike.com/system/avtnew/1674/16749385/thumb/2016101709394951.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119723053
文章简介: 


小民学习成绩渣渣，经常被老师带到办公室进行特别辅导。某次在办公室接受辅导时，老师拿着试卷对小民说道：“一改到你

##################################################
昵称: 历害吧
用户ID: 30975671
头像：http://pic.qiushibaike.com/system/avtnew/3097/30975671/thumb/20171028093230.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119716538
文章简介: None
['毕业季分手聚餐，酒喝到了一定程度，全班同学真情流露，各种表白，有跪的、有哭的、有笑的。。。', '身为班主任的我也深受感动，喝了不少，想着在倒下之前总要发表点临别赠言。。。', '于是开始了我有史以来最激情澎湃的演说，为了制造气势，说一句话摔一个盘子。。。', '当我摔到第四个盘子的时候，漂亮美丽的团支书一把抱住了我，我的小心脏瞬间从160飚到200+。。。', '只听她哭道，“老师，别摔了，盘子要钱的。”']
好笑: 1237
评论数: 31
##################################################
昵称: 喝醉酒的喵※
用户ID: 24703920
头像：http://pic.qiushibaike.com/system/avtnew/2470/24703920/thumb/20171026180612.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119732181
文章简介: None
['……今天，发小陪着我去相亲，相亲那男的是跳舞的，有颜值有身材，我心中暗喜。', '帅哥问我会跳钢管舞吗？发小抢着回答：她不会跳钢管舞，只会舞钢管，当年她一支钢管耍的可溜了，方圆几里的男孩都不敢惹她，，，']
好笑: 5895
评论数: 117
##################################################
昵称: °仙女界扛把子
用户ID: 10144082
头像：http://pic.qiushibaike.com/system/avtnew/1014/10144082/thumb/2017051306283654.JPEG?imageView2/1/w/90/h/90
文章详情ID: 119732665
文章简介: None
['一个特别聊得来的

##################################################
昵称: 带兵城管大队长
用户ID: 365566
头像：http://pic.qiushibaike.com/system/avtnew/36/365566/thumb/20160324121110.jpg?imageView2/1/w/90/h/90
文章详情ID: 119725666
文章简介: 


高中时候几个男同学合租一套房子。有一个特别色，特别骚，有一个公开女友，还隔三差五晚上去约小姑娘，夜不归宿。后来问他，你女朋友昨办，他说：骚得不行，玩玩而已。到了高中快毕业，果然开始闹分手，他女朋友的说：玩完我了就想甩了，门儿也没有。结果这哥们儿去北京上了个自考大学(参加高考就能上那种)，女孩也跟过去了。前段时间发请帖要结婚，新娘就是这个他嘴里"骚得不行"的女友。


['高中时候几个男同学合租一套房子。有一个特别色，特别骚，有一个公开女友，还隔三差五晚上去约小姑娘，夜不归宿。后来问他，你女朋友昨办，他说：骚得不行，玩玩而已。到了高中快毕业，果然开始闹分手，他女朋友的说：玩完我了就想甩了，门儿也没有。结果这哥们儿去北京上了个自考大学(参加高考就能上那种)，女孩也跟过去了。前段时间发请帖要结婚，新娘就是这个他嘴里"骚得不行"的女友。']
好笑: 420
评论数: 5
##################################################
昵称: 我的小冤家
用户ID: 12865517
头像：http://pic.qiushibaike.com/system/avtnew/1286/12865517/thumb/20160709231052.jpg?imageView2/1/w/90/h/90
文章详情ID: 119732327
文章简介: 


有一个初中女同学，长得人高马大的，初中毕业就上广州打工了。有一天下班路上被骑摩托车的抢包，她被拖了几米硬是没撒手，摩托车倒了，这货抢过包一通砸，后来那抢包的摩托车都没要就跑了……


['有一个初中女同学，长得人高马大的，初中毕业就上广州打工了。有一天下班路上被骑摩托车的抢包，她被拖了几米硬是没撒手，摩托车倒了，这货抢过包一通砸，后来那抢包的摩托车都没要就跑了……']
好笑: 6214
评论数: